In [1]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

con = sqlite3.connect('database.sqlite')

messages = pd.read_sql_query("""
SELECT Score, Summary,Text
FROM Reviews
WHERE Score != 3 order by Time
""", con)

def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

Score = messages['Score']
Score = Score.map(partition)
Text=messages['Text']
Text.head()

/home/varma/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/varma/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


0    this witty little book makes my son laugh at l...
1    I can remember seeing the show when it aired o...
2    Beetlejuice is a well written movie ..... ever...
3    A twist of rumplestiskin captured on film, sta...
4    A twist of rumplestiskin captured on film, sta...
Name: Text, dtype: object

# Train/Test Spliit

In [2]:
X_train,y_train=Text[:315488],Score[:315488]
X_cv,y_cv=Text[315488:420651],Score[315488:420651]
X_test,y_test=Text[420651:],Score[420651:]
print("Train Shape",X_train.shape)
print("Test Shape",X_test.shape)
print("Cross Validation Shape",X_cv.shape)

Train Shape (315488,)
Test Shape (105163,)
Cross Validation Shape (105163,)


# Converting into lowercase for train data

In [3]:
text_lst=X_train
text_lst=[item.encode('utf-8') for item in text_lst]
cX_train=[str(item).lower() for item in text_lst]
print (cX_train[0:2])

['b"this witty little book makes my son laugh at loud. i recite it in the car as we\'re driving along and he always can sing the refrain. he\'s learned about whales, india, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will still be able to recite from memory when he is  in college"', 'b"i can remember seeing the show when it aired on television years ago, when i was a child.  my sister later bought me the lp (which i have to this day,  i\'m thirty something).i used this series of books &amp; songs when i did my  student teaching for preschoolers &amp; turned the whole school on to it.  i am now purchasing it on cd, along with the books for my children 5 &amp;  2.  the tradition lives on!"']


In [4]:
text_lst=X_cv
text_lst=[item.encode('utf-8') for item in text_lst]
cX_cv=[str(item).lower() for item in text_lst]
print (cX_cv[0:2])

['b\'i\\\'ve ordered this coffee several times & it has been excellent every time.  it\\\'s not over-roasted/burnt tasting & the flavor is very complex.  the beans are oily, black & i found no quakers (underdeveloped coffee beans that make coffee taste awful - they stick out because they do not roast/darken properly).  that\\\'s a good test of coffee beans:  empty the bag on a cookie sheet & see how many quakers you find.  a few is "acceptable" but more than 10 is pretty bad in my opinion.  if you\\\'ve always been disappointed in dark roast coffee, you may want to give this a try but if you just don\\\'t like dark roast; you will probably not like this one.<br /><br />i use the "pour over" method to brew coffee every day because at this time, there is only one non-commercial drip coffeemaker that gets water to the ideal brewing temperature of 192f to 204 f.  i use cold, filtered water, a melitta natural brown filter, manual drip coffee filter & brew directly into my stainless carafe. 

# Converting into lowercase for Test data

In [5]:
text_test=X_test
text_test=[item.encode('utf-8') for item in text_test]
cX_test=[str(item).lower() for item in text_test]
print( cX_test[0:2])

['b"i was a little hesitant when i read it had a herbal taste , i like a conventional tasting coffee. it just arrived and i immediately brewed a pot. i definitely wasn\'t disappointed, i like this much more than the usual starbuck\'s french roast"', "b'buyer beware, the worst flavored hot cocoa i have ever tasted. never again would i waste my $ for this product, i tried different machine sizes and options, the rest goes in the garbage. what a rip off with two packages.'"]


# Removing Html tags from train data

In [6]:
import re

for i in range(len(cX_train)):
    cX_train[i]=re.sub(r'<.*?>', '', cX_train[i])
print (cX_train[0:2])

['b"this witty little book makes my son laugh at loud. i recite it in the car as we\'re driving along and he always can sing the refrain. he\'s learned about whales, india, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will still be able to recite from memory when he is  in college"', 'b"i can remember seeing the show when it aired on television years ago, when i was a child.  my sister later bought me the lp (which i have to this day,  i\'m thirty something).i used this series of books &amp; songs when i did my  student teaching for preschoolers &amp; turned the whole school on to it.  i am now purchasing it on cd, along with the books for my children 5 &amp;  2.  the tradition lives on!"']


In [7]:
import re

for i in range(len(cX_cv)):
    cX_cv[i]=re.sub(r'<.*?>', '', cX_cv[i])
print (cX_cv[0:2])

['b\'i\\\'ve ordered this coffee several times & it has been excellent every time.  it\\\'s not over-roasted/burnt tasting & the flavor is very complex.  the beans are oily, black & i found no quakers (underdeveloped coffee beans that make coffee taste awful - they stick out because they do not roast/darken properly).  that\\\'s a good test of coffee beans:  empty the bag on a cookie sheet & see how many quakers you find.  a few is "acceptable" but more than 10 is pretty bad in my opinion.  if you\\\'ve always been disappointed in dark roast coffee, you may want to give this a try but if you just don\\\'t like dark roast; you will probably not like this one.i use the "pour over" method to brew coffee every day because at this time, there is only one non-commercial drip coffeemaker that gets water to the ideal brewing temperature of 192f to 204 f.  i use cold, filtered water, a melitta natural brown filter, manual drip coffee filter & brew directly into my stainless carafe.  i heat the 

# Removing Html Tags from test data

In [8]:
for i in range(len(cX_test)):
    cX_test[i]=re.sub(r'<.*?>', '', cX_test[i])
print (cX_test[0:2])

['b"i was a little hesitant when i read it had a herbal taste , i like a conventional tasting coffee. it just arrived and i immediately brewed a pot. i definitely wasn\'t disappointed, i like this much more than the usual starbuck\'s french roast"', "b'buyer beware, the worst flavored hot cocoa i have ever tasted. never again would i waste my $ for this product, i tried different machine sizes and options, the rest goes in the garbage. what a rip off with two packages.'"]


# Removing Special Characters

In [9]:
for i in range(len(cX_train)):
    cX_train[i]=re.sub(r'[^a-zA-Z ]+', ' ', cX_train[i])
print (cX_train[0:2])

['b this witty little book makes my son laugh at loud  i recite it in the car as we re driving along and he always can sing the refrain  he s learned about whales  india  drooping roses   i love all the new words this book  introduces and the silliness of it all   this is a classic book i am  willing to bet my son will still be able to recite from memory when he is  in college ', 'b i can remember seeing the show when it aired on television years ago  when i was a child   my sister later bought me the lp  which i have to this day   i m thirty something i used this series of books  amp  songs when i did my  student teaching for preschoolers  amp  turned the whole school on to it   i am now purchasing it on cd  along with the books for my children    amp      the tradition lives on ']


In [10]:
for i in range(len(cX_cv)):
    cX_cv[i]=re.sub(r'[^a-zA-Z ]+', ' ', cX_cv[i])
print (cX_cv[0:2])

['b i ve ordered this coffee several times   it has been excellent every time   it s not over roasted burnt tasting   the flavor is very complex   the beans are oily  black   i found no quakers  underdeveloped coffee beans that make coffee taste awful   they stick out because they do not roast darken properly   that s a good test of coffee beans   empty the bag on a cookie sheet   see how many quakers you find   a few is  acceptable  but more than   is pretty bad in my opinion   if you ve always been disappointed in dark roast coffee  you may want to give this a try but if you just don t like dark roast  you will probably not like this one i use the  pour over  method to brew coffee every day because at this time  there is only one non commercial drip coffeemaker that gets water to the ideal brewing temperature of  f to   f   i use cold  filtered water  a melitta natural brown filter  manual drip coffee filter   brew directly into my stainless carafe   i heat the water to boiling  pour

In [11]:
for i in range(len(cX_test)):
    cX_test[i]=re.sub(r'[^a-zA-Z ]+', ' ', cX_test[i])
print (cX_test[0:2])

['b i was a little hesitant when i read it had a herbal taste   i like a conventional tasting coffee  it just arrived and i immediately brewed a pot  i definitely wasn t disappointed  i like this much more than the usual starbuck s french roast ', 'b buyer beware  the worst flavored hot cocoa i have ever tasted  never again would i waste my   for this product  i tried different machine sizes and options  the rest goes in the garbage  what a rip off with two packages ']


# Removing Stopwords

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopword=set(stopwords.words('english'))
for i in range(len(cX_train)):
    text_stoprm=[]
    text_tokens=[]
    text_tokens=cX_train[i].split()
    for r in text_tokens:
        if not r in stopword:
            text_stoprm.append(r)
    cX_train[i]=' '.join(text_stoprm)
    
print (cX_train[0:2])

['b witty little book makes son laugh loud recite car driving along always sing refrain learned whales india drooping roses love new words book introduces silliness classic book willing bet son still able recite memory college', 'b remember seeing show aired television years ago child sister later bought lp day thirty something used series books amp songs student teaching preschoolers amp turned whole school purchasing cd along books children amp tradition lives']


In [13]:
for i in range(len(cX_cv)):
    text_stoprm=[]
    text_tokens=[]
    text_tokens=cX_cv[i].split()
    for r in text_tokens:
        if not r in stopword:
            text_stoprm.append(r)
    cX_cv[i]=' '.join(text_stoprm)
    
print (cX_cv[0:2])

['b ordered coffee several times excellent every time roasted burnt tasting flavor complex beans oily black found quakers underdeveloped coffee beans make coffee taste awful stick roast darken properly good test coffee beans empty bag cookie sheet see many quakers find acceptable pretty bad opinion always disappointed dark roast coffee may want give try like dark roast probably like one use pour method brew coffee every day time one non commercial drip coffeemaker gets water ideal brewing temperature f f use cold filtered water melitta natural brown filter manual drip coffee filter brew directly stainless carafe heat water boiling pour tablespoons grounds saturate plump beans wait minute flood grounds rest water lot crema beautiful reddish brown foam develops top beans smells tastes wonderful brew directly carafe using coffee cone pour method rsvp manual drip coffee filter cone carafes thermosi use grinder bodum bistro electric burr coffee grinder blacki use water kettle heat water ost

In [14]:
for i in range(len(cX_test)):
    text_stoprm=[]
    text_tokens=[]
    text_tokens=cX_test[i].split()
    for r in text_tokens:
        if not r in stopword:
            text_stoprm.append(r)
    cX_test[i]=' '.join(text_stoprm)
    
print( cX_test[0:2])

['b little hesitant read herbal taste like conventional tasting coffee arrived immediately brewed pot definitely disappointed like much usual starbuck french roast', 'b buyer beware worst flavored hot cocoa ever tasted never would waste product tried different machine sizes options rest goes garbage rip two packages']


# Stemming Data

In [15]:
from nltk.stem.snowball import SnowballStemmer
stemmer =SnowballStemmer('english')

for i in range(len(cX_train)):
    text_stem=[]
    text_tokens=[]
    text_tokens=cX_train[i].split()
    for r in text_tokens:
        text_stem.append(str(stemmer.stem(r)))
    cX_train[i]=' '.join(text_stem)
    


In [16]:

for i in range(len(cX_cv)):
    text_stem=[]
    text_tokens=[]
    text_tokens=cX_cv[i].split()
    for r in text_tokens:
        text_stem.append(str(stemmer.stem(r)))
    cX_cv[i]=' '.join(text_stem)
    

In [17]:
from nltk.stem.snowball import SnowballStemmer
stemmer =SnowballStemmer('english')

for i in range(len(cX_test)):
    text_stem=[]
    text_tokens=[]
    text_tokens=cX_test[i].split()
    for r in text_tokens:
        text_stem.append(str(stemmer.stem(r)))
    cX_test[i]=' '.join(text_stem)
    


In [18]:
tfidfvec=TfidfVectorizer()
tfidf_X_train=tfidfvec.fit_transform(cX_train)

In [19]:
tfidf_X_train.shape

(315488, 65308)

In [20]:
tfidf_X_cv=tfidfvec.transform(cX_cv)
tfidf_X_cv.shape

(105163, 65308)

In [21]:
tfidf_X_test=tfidfvec.transform(cX_test)
tfidf_X_test.shape

(105163, 65308)

# Logistic Regression 

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
alpha=[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]

maxval=0
bestalpha=0
for i in alpha:
    logistic_clf=LogisticRegression(C=i)
    logistic_clf.fit(tfidf_X_train,y_train)
    cvpredicted=logistic_clf.predict(tfidf_X_cv)
    score=accuracy_score(y_cv,cvpredicted)
    print('for alpha',i,'accuracy score is',score)
    if(maxval<score):
        maxval=score
        bestalpha=i
        
print('-'*100)
print('the best alpha is ',bestalpha,'and score on CV is',maxval)

print('-'*100)

#for test data
clf=LogisticRegression(C=bestalpha)
clf.fit(tfidf_X_train,y_train)

trainpredicted=clf.predict(tfidf_X_train)
score=accuracy_score(y_train,trainpredicted)
print('train accuracy is',score)

cvpredicted=clf.predict(tfidf_X_cv)
score=accuracy_score(y_cv,cvpredicted)
print('cv accuracy',score)

testpredicted=clf.predict(tfidf_X_test)
score=accuracy_score(y_test,testpredicted)
print('test accuracy',score)


for alpha 1e-05 accuracy score is 0.822237859323146
for alpha 0.0001 accuracy score is 0.822237859323146
for alpha 0.001 accuracy score is 0.822237859323146
for alpha 0.01 accuracy score is 0.8383652044920742
for alpha 0.1 accuracy score is 0.9024752051577075
for alpha 1 accuracy score is 0.91816513412512
for alpha 10 accuracy score is 0.9159114897825281
for alpha 100 accuracy score is 0.907106111465059
for alpha 1000 accuracy score is 0.9019807346690376
----------------------------------------------------------------------------------------------------
the best alpha is  1 and score on CV is 0.91816513412512
----------------------------------------------------------------------------------------------------
train accuracy is 0.9405460746526016
cv accuracy 0.91816513412512
test accuracy 0.9152173292888183


# Linear SVM

In [24]:
from sklearn.linear_model import SGDClassifier

alpha=[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]

maxval=0
bestalpha=0
for i in alpha:
    svm_clf=SGDClassifier(alpha=i,loss='hinge')
    svm_clf.fit(tfidf_X_train,y_train)
    cvpredicted=svm_clf.predict(tfidf_X_cv)
    score=accuracy_score(y_cv,cvpredicted)
    print('for alpha',i,'accuracy score is',score)
    if(maxval<score):
        maxval=score
        bestalpha=i
        
print('-'*100)
print('the best alpha is ',bestalpha,'and score on CV is',maxval)

print('-'*100)

#for test data
clf=SGDClassifier(alpha=bestalpha,loss='hinge')
clf.fit(tfidf_X_train,y_train)

trainpredicted=clf.predict(tfidf_X_train)
score=accuracy_score(y_train,trainpredicted)
print('train accuracy is',score)

cvpredicted=clf.predict(tfidf_X_cv)
score=accuracy_score(y_cv,cvpredicted)
print('cv accuracy',score)

testpredicted=clf.predict(tfidf_X_test)
score=accuracy_score(y_test,testpredicted)
print('test accuracy',score)


/home/varma/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


for alpha 1e-05 accuracy score is 0.9169860121905994
for alpha 0.0001 accuracy score is 0.894554168291129
for alpha 0.001 accuracy score is 0.8222758955145821
for alpha 0.01 accuracy score is 0.822237859323146
for alpha 0.1 accuracy score is 0.822237859323146
for alpha 1 accuracy score is 0.822237859323146
for alpha 10 accuracy score is 0.822237859323146
for alpha 100 accuracy score is 0.822237859323146
for alpha 1000 accuracy score is 0.822237859323146
----------------------------------------------------------------------------------------------------
the best alpha is  1e-05 and score on CV is 0.9169860121905994
----------------------------------------------------------------------------------------------------
train accuracy is 0.9417568972512426
cv accuracy 0.9169289579034451
test accuracy 0.9141713340243242
